# Klasyfikacja za pomocą algorytmu wektorów wspierających (SVM)
## Materiały
Na tych ćwiczeniach zapoznamy się z zastosowaniem SVM do klasyfikacji.
Notebook bazuje na tutorialu w [Medium](https://medium.com/swlh/visualizing-svm-with-python-4b4b238a7a92).

In [ ]:
import numpy as np
import matplotlib.pylab as plt

#### Importujemy dane uczące, znów iryski, ale tym razem skorzystamy do importu z biblioteki Seaborn.
Daje nam to dodatkowe wygodne narzędzia jak "drop", "pairplot", ale też zmienia format danych, jako że korzysta z frameworku 'pandas'. Nie utrudnia to znacząco życia, ale czasem wymaga dodatkowej uwagi. Będę ostrzegać :)

In [ ]:
import seaborn as sns
iris = sns.load_dataset("iris")
print(iris.head())
y = iris.species
X = iris.drop('species',axis=1)
sns.pairplot(iris, hue="species",palette="bright")

## Liniowo separowalne dane

#### Ograniczamy się dziś do dwóch cech i dwóch klas irysów. 

W pierwszej części ćwiczeń chciałabym wizualizować nasze wyniki, by złapać podstawową intuicję za działaniem SVM. Wyłącznie w celu ułatwienia wizualizacji ograniczymy się więc na razie do dwóch cech i dwóch klas. Np. weźmy cechy petal_width, petal_length oraz klasy Setosa i Versicolor.
    

In [ ]:
data = iris[(iris['species']!='virginica')]
data = data.drop(['sepal_length','sepal_width'], axis=1)
data.head()

Zmienimy teraz nazwy kategorii na liczby i zwizualizujemy, na czym będziemy dzisiaj pracować :)

In [ ]:
data = data.replace('setosa', 0)
data = data.replace('versicolor', 1)
X = data.iloc[:,0:2] # the 'iloc' to właśnie konsekwencja korzystania z pandas. Gdyby data było zwykłym arrayem, starczyłoby data[:,0:2] :)
y = data['species']
plt.scatter(X.iloc[:, 0], X.iloc[:, 1], c=y, s=50, cmap='autumn')

Czas na trening modelu. Jak może już zauważyliście, ten etap w sklearn jest bardzo podobny niezależnie od modelu. Analogicznie wyglądał dla naiwnego Bayesa!

In [ ]:
from sklearn.svm import SVC 
model = SVC(kernel='linear', C=1E10)
model.fit(X, y)

Po raz pierwszy na ćwiczeniach dotykamy tematu regularyzacji modelu. Regularyzacja to ogólnie rzecz biorąc każda technika, która zwiększa zdolność generalizacji modelu. Będziemy o tym mówić szczególnie w kontekście sieci neuronowych. Tutaj takim parametrem regularyzacyjnym jest parametr C, regulujący koszt "naruszania" marginesów zbudowanych przez model. Im mniejszy parametr C, tym mniejszy koszt i większa szansa, że jakiś

#### Możemy zwizualizować wekstory wspierające, na których zbudowany jest nasz model

In [ ]:
print(model.support_vectors_)

plt.scatter(X.iloc[:, 0], X.iloc[:, 1], c=y, s=50, cmap='autumn')
plt.scatter(model.support_vectors_[:,0],model.support_vectors_[:,1])

#### Narysujmy te dane

In [ ]:
ax = plt.gca()

# Rysujemy wszystkie dane treningowe
plt.scatter(X.iloc[:, 0], X.iloc[:, 1], c=y, s=50, cmap='autumn')

# Budujemy siatkę (meshgrid)
xlim = ax.get_xlim()
ylim = ax.get_ylim()
xx = np.linspace(xlim[0], xlim[1], 30)
yy = np.linspace(ylim[0], ylim[1], 30)
YY, XX = np.meshgrid(yy, xx)
xy = np.vstack([XX.ravel(), YY.ravel()]).T

# Rysujemy funkcję decyzyjną nauczonego modelu
Z = model.decision_function(xy).reshape(XX.shape)
ax.contour(XX, YY, Z, colors='k', levels=[-1, 0, 1], alpha=0.5,
           linestyles=['--', '-', '--'])

# Dorysowywujemy wektory wspierające
support_vector1 = model.support_vectors_[:, 0]
support_vector2 = model.support_vectors_[:, 1]
ax.scatter(support_vector1, support_vector2, s=100,
           linewidth=1, facecolors='none', edgecolors='k')

plt.show()

#### Co się stanie z wyznaczoną funkcją decyzji jak usuniemy część danych uczących?

Korzystając z kawałka już napisanego kodu, który buduje Wam dane uczące biorąc wszystkie dane z żółtej klasy versicolor i losuje 70% danych z czerwonej klasy setosa, przetrenujcie kilka razy SVM, rysując funkcję decyzji (jak wyżej). Jakie macie wnioski?

PS. Najlepiej zróbcie sobie z tego funkcję z argumentami: X, y, kernel, C, gamma, bo będziecie tego używać jeszcze kilka razy :)

In [ ]:
def trainSVM_and_visualize(X_train, Y_train, kernel, C=1.0, gamma='scale'):
  # Trening modelu
  ...

  # Deklarujemy obiekt wykres
  ...

  # Rysujemy wszystkie dane treningowe
  ...

  # Budujemy siatkę (meshgrid)
  ...

  # Rysujemy funkcję decyzyjną nauczonego modelu
  ...

  # Dorysowywujemy wektory wspierające
  ...

  plt.show()

In [ ]:
# Przygotowanie danych uczących
red_sample = data.sample(frac=0.7)
X = red_sample.iloc[:,0:2]
y = red_sample['species']

trainSVM_and_visualize(X, y, 'linear')

## Liniowo nieseparowalne dane

Aby zwizualizować super moc SVM na danych nieseparowalnych liniowo, to zaczniemy od sztucznie wygenerowanych danych.

In [ ]:
from sklearn.datasets.samples_generator import make_circles
X, y = make_circles(100, factor=.1, noise=.1)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')

#### Wytrenujcie na takich danych SVM z liniowym kernelem i zwizualizujcie wyniki
Proponuję użyć do tego tej samej funkcji co stworzyliście ostatnio, ALE dodajcie dodatkowy argument opcjonalny np. 'panda = True' od którego zależeć będzie linijka z plotowaniem wszystkich danych (czyli plt.scatter...). Do tej pory korzystaliście z danych pandowych, więc używaliście 'iloc', teraz wracamy do array'ów zwykłych, więc trzeba iloca się pozbyć :) proponuję zrobić tam if'a.

In [ ]:
trainSVM_and_visualize(X, y, 'linear', panda="False")

### Co zrobi nam za to SVM jak ustalimy kernel np. gaussowski?
Możemy wyobrazić sobie, że te dane są nieźle separowalne, jak zmapujemy je odpowiednią funkcją w wyższy wymiar. Np:

In [ ]:
from mpl_toolkits import mplot3d

# wchodzimy w 3D za pomocą RBF scentrowanym w środku żółtego bloba
r = np.exp(-(X ** 2).sum(1))
ax = plt.subplot(projection='3d')
ax.scatter3D(X[:, 0], X[:, 1], r, c=y, s=50, cmap='autumn')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('r')

Teraz zrobiliśmy to ręcznie, gdybyśmy chcieli podejść do tego na poważnie, to musielibyśmy zastanowić się nad optymalnym mapowaniem, które najlepiej dzieli klasy, a także nad wydajnością numeryczną takiego mapowania. Wszystko to siedzi w środku SVM, z naszej strony wymaga ona tylko zastanowienia się jakiego kernela chcemy użyć. Polecam wizualizacje pod [tym linkiem](https://www.kaggle.com/joparga3/3-visualising-how-different-kernels-in-svms-work), które dają fajną intuicję czego po różnych kernelach oczekiwać. Ostatecznie jednak warto sprawdzić samą funkcję mapującą.

### Użyjcie teraz kernela = 'rbf'

In [ ]:
trainSVM_and_visualize(X, y, 'rbf', panda="False")

## Zadanie na dzisiaj
Czym są parametry C i gamma w definicji modelu SVM? Proszę znaleźć i opisać za co odpowiadają. Ponadto zadaniem na dzisiaj będzie dobranie optymalnych C i gamma (hiperparametrów). Zanim wyrobi się intuicję czego i jak szukać, można stosować (drogie) podejście systematyczne. Czyli wykonujemy wiele treningów dla ustalonych wartości C i gamma, po czym wyliczamy metryki klasyfikacji dla każdej kombinacji i wizualizujemy najlepsze. To jest zadanie na dzisiaj :)

### Będziecie pracować znów na iryskach i znów na dwóch cechach (żeby ułatwić wizualizację), ale tym razem weźmy trzy klasy.

In [ ]:
iris = sns.load_dataset("iris")
print(iris.head())

y = iris.species
y = y.replace('setosa', 0)
y = y.replace('versicolor', 1)
y = y.replace('virginica', 2)

X = iris.drop('species',axis=1)
X = X.drop(['sepal_length','sepal_width'], axis=1)
X.head()

Koniec z czystą wizualizacją, teraz na poważnie dobieramy hiperparametry i oceniamy jakość klasyfikacji. Potrzebujemy podziału na dane treningowe i testowe! (np. za pomocą *train_test_split* z *sklearn.model_selection*)

In [ ]:
X_train, X_test, y_train, y_test = ...

Mając taki podział danych możemy dopasować model SVM do części uczącej. Mamy teraz jednak trzy klasy, a decision_function rysuje się dla jednej klasy vs. pozostałe osobno (mielibyśmy chaotyczny rysunek), więc lepiej zmodyfikować część wizualizacyjną i zamiast tego robić predykcje dla każdego elementu mesha (tak jak na dwóch zajęciach temu, czyli Bayes na iryskach).

In [ ]:
def trainSVM_and_visualize_multiclass(X_train, Y_train, kernel, C=1.0, gamma='scale')
  ...

  # Deklaracja obiektu wykres
  ...

  # Rysujemy wszystkie dane treningowe
  ...

  # Budujemy siatkę (meshgrid)
  h = 0.02 # krok w meshu
  x_min, x_max = X.iloc[:, 0].min() - 1, X.iloc[:, 0].max() + 1
  y_min, y_max = X.iloc[:, 1].min() - 1, X.iloc[:, 1].max() + 1
  xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

  # Robimy predykcję dla każdego elementu meshgridu
  Z = model.predict(np.c_[xx.ravel(), yy.ravel()])

  # Predykcje zmieniamy w kolory
  Z = Z.reshape(xx.shape)
  plt.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.8)

  # Dorysowywujemy wektory wspierające
  ...

  plt.xlim(xx.min(), xx.max())
  plt.ylim(yy.min(), yy.max())
  plt.show()

In [ ]:
trainSVM_and_visualize_multiclass(X_train, y_train, kernel='rbf')

Teraz możemy zastosować miary jakości klasyfikacji.

In [ ]:
...

#### Proszę napisać kod, który
* skanuje przestrzeń (C, gamma): C w zakresie od 0.1 do 100, gamma w zakresie od 0.1 do 10. Do wygenerowania zakresu ze skalą logarytmiczną można wykorzystać np. takie polecenie: <tt>zakresC = np.logspace(np.log2(0.1),np.log2(100), 8, base=2)</tt>
* znajduje najlepsze parametry
* rysuje podział przestrzeni dla najlepszych parametrów.

Którą miarę jakości będziecie maksymalizować?

Stąd możecie czerpać inspirację: [link](https://scikit-learn.org/stable/auto_examples/svm/plot_rbf_parameters.html).